In [137]:
import warnings
from statsbombpy.api_client import NoAuthWarning
warnings.simplefilter('ignore', NoAuthWarning)
from socceraction.data.statsbomb import StatsBombLoader

api = StatsBombLoader(getter="remote", creds={"user": "", "passwd": ""})

In [138]:
import random
import math
import matplotsoccer
import socceraction.spadl as spadl
from tabulate import tabulate
import pandas as pd
import numpy as np

def nice_time(row):
    minute = int((row.period_id-1)*45 +row.time_seconds // 60)
    second = int(row.time_seconds % 60)
    return f"{minute}m{second}s"

# Function to calculate the Euclidean distance between two points
def calculate_distance(start_location, end_location):
    distance = np.round(math.sqrt((end_location[0] - start_location[0])**2 + (end_location[1] - start_location[1])**2),2)

    return "{:.2f}".format(distance)

def isProgressive(start_location, end_location):
    
    if (end_location[1] - start_location[1]) > 1:
        return True
    else:
        return False


df_competitions = api.competitions()
random_row = df_competitions.sample(1)

competition_id = random_row['competition_id'].item()
season_id = random_row['season_id'].item()
competition_name = random_row['competition_name'].item()
season_name = random_row['season_name'].item()

df_games = api.games(competition_id=competition_id, season_id=season_id)

all_teams = list(df_games['home_team_id'].unique())
team_id = random.choice(all_teams)

df_games = df_games[(df_games['away_team_id'] == team_id) | (df_games['home_team_id'] == team_id)]
all_games = list(df_games['game_id'])
g = random.choice(all_games)

c_stage = df_games[df_games['game_id'] == g]['competition_stage'].item()
df_teams = api.teams(game_id=g)
df_players = api.players(game_id=g)
df_events = api.events(game_id=g)
team_name = df_events[df_events['team_id'] == team_id]['team_name'].iloc[0]
teams = list(df_events['team_name'].unique())
teams.remove(team_name)

c_string = f"{team_name} vs {teams[0]} ({competition_name} {c_stage} - {season_name})"
print(c_string)

carries_df = df_events[df_events["type_name"] == 'Carry'].copy()
carries_df['distance_carried'] = carries_df.apply(lambda row: calculate_distance(row['location'], row['extra']['carry']['end_location']), axis=1)
carries_df['isProgressive'] = carries_df.apply(lambda row: isProgressive(row['location'], row['extra']['carry']['end_location']), axis=1)
carries_df = carries_df[(carries_df['under_pressure'] == True) & (carries_df['isProgressive'] == True)]

sum_distance_carried = carries_df.groupby('player_name')['distance_carried'].sum().sort_values(ascending=False).head(5)

carries_df = carries_df[carries_df['player_name'] == random.choice(list(sum_distance_carried.index))][['minute', 'second', 'player_name', 'play_pattern_name', 'duration', 'distance_carried']]

print(tabulate(carries_df, headers='keys', tablefmt='psql'))



Albacete vs Barcelona (La Liga Regular Season - 2004/2005)
+------+----------+----------+--------------------+---------------------+------------+--------------------+
|      |   minute |   second | player_name        | play_pattern_name   |   duration |   distance_carried |
|------+----------+----------+--------------------+---------------------+------------+--------------------|
|  750 |       19 |       48 | Samuel Eto''o Fils | From Corner         |   0.4738   |               4.6  |
| 2118 |       55 |       49 | Samuel Eto''o Fils | Regular Play        |   0.778042 |               1.12 |
| 2471 |       65 |       42 | Samuel Eto''o Fils | Regular Play        |   3.38954  |              16.1  |
| 2860 |       76 |       16 | Samuel Eto''o Fils | From Free Kick      |   1.517    |              11.5  |
| 3019 |       79 |       21 | Samuel Eto''o Fils | Regular Play        |   1.68421  |               8.37 |
+------+----------+----------+--------------------+---------------------+----